In [1]:
! git clone "https://github.com/gdbwoo/DS-3001-Projects"

Cloning into 'DS-3001-Projects'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 139 (delta 8), reused 0 (delta 0), pack-reused 120
Receiving objects: 100% (139/139), 11.58 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
# Import all necessary packages
import geopandas as gpd
#import mapclassify (fix this)
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
# Load data
#gdf = gpd.read_file('DS-3001-Projects/Project 3/VirginiaCounty_ClippedToShoreline.shx') # Load Virginia shapefile for maps (fix this)
vdf = pd.read_csv('DS-3001-Projects/Project 3/voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('DS-3001-Projects/Project 3/county_adjacencies.csv') # Load county adjacency data and population data

In [ ]:
# # Merge shapefile data and county adjacency data:
# gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS'])
# df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')  # merge county population and adjacency data into df
# del gdf, cdf